In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
import pandas as pd
from time import sleep

In [2]:
url = 'https://www.thegioididong.com/tui-chong-soc#c=7923&o=7&pi=3'

driver = webdriver.Chrome()
driver.get(url)

### Lấy danh sách sản phẩm

In [4]:
listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
len(product_list)

63

#### Tạo Func lấy thông 1 tin sản phẩm

In [3]:
def get_product_info(product_elm):
    phone_name = product_elm.find_element(By.XPATH, './/h3').text

    try:
        phone_price = product_elm.find_element(By.XPATH, './/strong[@class="price"]').text
    except:
        phone_price = "N/A"

    
    if phone_price == "N/A":
        try:
            phone_price = product_elm.find_element(By.CLASS_NAME, 'box-price-present').text
        except:
            phone_price = "N/A"
        
    result = {
        'name': phone_name,
        'price': phone_price
    }
    return result

In [5]:
# Hàm lấy review của tất cả điện thoại
all_reviews = []
tmp = 0
def getReviewAllPhone(product_list):
    for item in product_list:
        product_info = get_product_info(item)
        item.click()
        sleep(2)

        try:
            # Nếu mà có readmore thì mình bấm luôn
            # read_more_elm = driver.find_element(By.XPATH, '/html/body/section/div[2]/div[1]/div[11]/div[2]/div/div/div[6]/div/a')
            read_more_elm = driver.find_element(By.CLASS_NAME, 'btn-view-all')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        try:
                            all_reviews.append({
                                'product_name': product_info['name'],
                                'product_price': product_info['price'],
                                'review': review.text
                            })
                        except:
                            continue

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None # Init var
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page: # if next_page exist
                            next_page.click() # forward
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            # Lấy xong hết review của trang cuối cùng của sản phẩm
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                try:
                                    all_reviews.append({
                                        'product_name': product_info['name'],
                                        'product_price': product_info['price'],
                                        'review': review.text
                                    })
                                except:
                                    continue
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:

            # Nếu có review nhưng chỉ có ===>> 1 TRANG
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            # Lấy xong thì back về
            driver.back()
            sleep(2)
            listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
            product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'ajaxed')

In [7]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep

all_reviews = []
tmp = 0

def getReviewAllPhone_v02(product_list):
    global tmp
    i = 0  # Bắt đầu từ 64
    while i < len(product_list):
        # Lấy thông tin sản phẩm và click vào sản phẩm
        product_info = get_product_info(product_list[i])
        item = product_list[i]
        item.click()
        sleep(2)

        try:
            # Nếu có nút "Xem thêm" thì bấm
            read_more_elm = driver.find_element(By.CLASS_NAME, 'btn-view-all')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        try:
                            all_reviews.append({
                                'product_name': product_info['name'],
                                'product_price': product_info['price'],
                                'review': review.text
                            })
                        except:
                            continue

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page:
                            next_page.click()
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                try:
                                    all_reviews.append({
                                        'product_name': product_info['name'],
                                        'product_price': product_info['price'],
                                        'review': review.text
                                    })
                                except:
                                    continue
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:
            # Nếu chỉ có 1 trang review
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            driver.back()
            sleep(2)

        # Sau khi quay lại, làm mới product_list
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'listproduct'))
        )
        listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
        product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
        print(f"Độ dài product_list sau khi làm mới: {len(product_list)}")
        
        i += 1  # Tăng i để xử lý sản phẩm tiếp theo

In [8]:
getReviewAllPhone_v02(product_list)

Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 63
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 63
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 63
Độ dài product_list sau khi làm mới: 6

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6E7C6C085+28581]
	(No symbol) [0x00007FF6E7BD43C0]
	(No symbol) [0x00007FF6E7A691DA]
	(No symbol) [0x00007FF6E7ABF797]
	(No symbol) [0x00007FF6E7ABF9CC]
	(No symbol) [0x00007FF6E7B134C7]
	(No symbol) [0x00007FF6E7AE797F]
	(No symbol) [0x00007FF6E7B10193]
	(No symbol) [0x00007FF6E7AE7713]
	(No symbol) [0x00007FF6E7AB0400]
	(No symbol) [0x00007FF6E7AB16B3]
	GetHandleVerifier [0x00007FF6E7FC2D2D+3529805]
	GetHandleVerifier [0x00007FF6E7FDB9E2+3631362]
	GetHandleVerifier [0x00007FF6E7FD0253+3584371]
	GetHandleVerifier [0x00007FF6E7D36D5A+859258]
	(No symbol) [0x00007FF6E7BDF12F]
	(No symbol) [0x00007FF6E7BDAF94]
	(No symbol) [0x00007FF6E7BDB136]
	(No symbol) [0x00007FF6E7BCAB09]
	BaseThreadInitThunk [0x00007FFEBF53E8D7+23]
	RtlUserThreadStart [0x00007FFEBFE5BF6C+44]


In [9]:
all_reviews

[{'product_name': 'Túi chống sốc Laptop 15.6 inch Togo kéo ngang',
  'product_price': '120.000₫',
  'review': 'Sản phẩm tốt'},
 {'product_name': 'Túi chống sốc Laptop 15.6 inch Togo kéo ngang',
  'product_price': '120.000₫',
  'review': 'Oke'},
 {'product_name': 'Túi chống sốc Laptop 15.6 inch Togo kéo ngang',
  'product_price': '120.000₫',
  'review': 'Túi cứng cáp chắc chắn'},
 {'product_name': 'Túi chống sốc Laptop 15.6 inch Togo kéo ngang',
  'product_price': '120.000₫',
  'review': 'kích thước chưa đúng lắm'},
 {'product_name': 'Túi chống sốc Laptop 15.6 inch Togo kéo ngang',
  'product_price': '120.000₫',
  'review': 'Ko bỏ vừa laptop 15.6 inCo nhet vao van Dư 1 đoạn'},
 {'product_name': 'Túi chống sốc Laptop 15.6 inch Togo kéo ngang',
  'product_price': '120.000₫',
  'review': 'Sản phẩm ghi laptop 15.6 inch nhưng để vào thì laptop thừa ra hẳn 1 đoạn, túi vô ích'},
 {'product_name': 'Túi chống sốc Laptop 15.6 inch Togo kéo ngang',
  'product_price': '120.000₫',
  'review': 'Tạm ổ

In [10]:
df = pd.DataFrame(all_reviews)
# Lưu file
df.to_csv("E:\\study\\tu_hoc\\DE\\CRAWL_DATA\\review_data\\data\\backpack_data.xlsx", index=False)